# Phosphate Trafficking

Model is from Chapter 8 of Systems Biology: Simulation of Dynamic Network States.

Bernhard Ø. Palsson. Systems Biology: Simulation of Dynamic Network States. Cambridge University Press, 2011. doi:10.1017/CBO9780511736179.

In [1]:
from os import path

from mass import MassMetabolite, MassModel, MassReaction, UnitDefinition
from mass.io import json

## Model

In [2]:
model = MassModel("Phosphate_Trafficking")

Academic license - for non-commercial use only


## Metabolites

In [3]:
atp_c = MassMetabolite(
    "atp_c",
    name="ATP",
    formula="C10H12N5O13P3",
    charge=-4,
    compartment="c")

adp_c = MassMetabolite(
    "adp_c",
    name="ADP",
    formula="C10H12N5O10P2",
    charge=-3,
    compartment="c")

amp_c = MassMetabolite(
    "amp_c",
    name="AMP",
    formula="C10H12N5O7P",
    charge=-2,
    compartment="c")

b = MassMetabolite(
    "B",
    name="Phosphagen buffer (Free)",
    formula="[B]",
    charge=0,
    compartment="c")

bp = MassMetabolite(
    "BP",
    name="Phosphagen buffer (Loaded)",
    formula="[B]-PO3",
    charge=-1,
    compartment="c")

### Initial Conditions

In [4]:
atp_c.ic = 1.7
adp_c.ic = 0.2
amp_c.ic = 0.2
b.ic = 2
bp.ic = 8

### Compartments

In [5]:
model.compartments = {"c": "compartment"}

## Reactions

In [6]:
distr = MassReaction("distr", name="Distribution", reversible=True)
use = MassReaction("use", name="ATP Utilization", reversible=False)
form = MassReaction("form", name="ATP Formation", reversible=False)
buffer = MassReaction("buffer", name="ATP Buffering")

distr.add_metabolites({
    adp_c: -2,
    atp_c: 1,
    amp_c: 1})

use.add_metabolites({
    atp_c: -1,
    adp_c: 1})

form.add_metabolites({
    adp_c: -1,
    atp_c: 1})

buffer.add_metabolites({
    b: -1,
    atp_c: -1,
    adp_c: 1,
    bp: 1})

model.add_reactions([distr, use, form, buffer])

amp_drain = model.add_boundary(
    amp_c,
    boundary_type="demand",
    reaction_id="amp_drain")

amp_in = model.add_boundary(
    amp_c,
    boundary_type="demand",
    reaction_id="amp_in")
amp_in.reverse_stoichiometry(inplace=True)

model

Name,Phosphate_Trafficking
Memory address,0x011d123c90
Stoichiometric Matrix,5x6
Matrix Rank,4
Number of metabolites,5
Initial conditions defined,5/5
Number of reactions,6
Number of genes,0
Number of enzyme modules,0
Number of groups,0
Objective expression,0


### Parameters and Rates

In [7]:
distr.kf = 1000
distr.Keq = 1

use.kf = 6.25

form.kf = 25

buffer.kf = 1000
buffer.Keq = 1

amp_drain.kf = 0.3

model.add_custom_rate(amp_in, custom_rate="b1", custom_parameters={"b1": 0.03})

## Units

In [8]:
concentration_unit = UnitDefinition(id="mM", name="Millimolar")
concentration_unit.create_unit(kind="mole", exponent=1, scale=-3, multiplier=1)
concentration_unit.create_unit(kind="liter", exponent=-1, scale=1, multiplier=1)

time_unit = UnitDefinition(id="min", name="Minute")
time_unit.create_unit(kind="second", exponent=1, scale=1, multiplier=60)
model.add_units([concentration_unit, time_unit])

## Export

In [9]:
json.save_json_model(
    mass_model=model,
    filename=path.realpath(path.join("models", "Phosphate_Trafficking.json")))